In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import tkinter as tk
from tkinter import messagebox
import numpy as np

# Updated file paths
red_wine_path = r"C:\Users\user\Music\wine dataset\red_wine.csv"
white_wine_path = r"C:\Users\user\Music\wine dataset\white_wine.csv"

# Define feature ranges for validation
feature_ranges = {
    "fixed acidity": (4.0, 15.0),
    "volatile acidity": (0.1, 1.5),
    "citric acid": (0.0, 1.0),
    "residual sugar": (0.0, 15.0),
    "chlorides": (0.0, 0.2),
    "free sulfur dioxide": (0, 100),
    "total sulfur dioxide": (0, 300),
    "density": (0.99, 1.05),
    "pH": (2.5, 4.0),
    "sulphates": (0.3, 1.0),
    "alcohol": (8.0, 15.0)
}

# Initialize models
red_model, white_model = None, None
red_accuracy, white_accuracy = None, None
red_pred_errors, white_pred_errors = None, None
red_means, white_means = None, None

def train_models():
    global red_model, white_model, red_accuracy, white_accuracy, red_pred_errors, white_pred_errors, red_means, white_means
    def preprocess_data(file_path):
        data = pd.read_csv(file_path)
        data.columns = data.columns.str.strip()
        X, y = data.drop("quality", axis=1), data["quality"]
        return train_test_split(X, y, test_size=0.2, random_state=42), X.mean()
    
    (X_train_red, X_test_red, y_train_red, y_test_red), red_means = preprocess_data(red_wine_path)
    (X_train_white, X_test_white, y_train_white, y_test_white), white_means = preprocess_data(white_wine_path)
    
    red_model, white_model = LinearRegression(), LinearRegression()
    red_model.fit(X_train_red, y_train_red)
    white_model.fit(X_train_white, y_train_white)
    
    red_accuracy, white_accuracy = red_model.score(X_test_red, y_test_red), white_model.score(X_test_white, y_test_white)
    red_pred_errors, white_pred_errors = np.abs(red_model.predict(X_test_red) - y_test_red), np.abs(white_model.predict(X_test_white) - y_test_white)

def predict_quality(wine_type, input_features):
    try:
        means = red_means if wine_type == "red" else white_means
        filled_features = [float(input_features[i]) if input_features[i] else means[i] for i in range(len(input_features))]
        model, accuracy, pred_errors = (red_model, red_accuracy, red_pred_errors) if wine_type == "red" else (white_model, white_accuracy, white_pred_errors)
        
        if model is None:
            messagebox.showerror("Model Error", "Model is not trained. Please restart the application.")
            return
        
        prediction = model.predict([filled_features])[0]
        quality_classification = "Low Quality" if prediction <= 5 else "Medium Quality" if prediction <= 7 else "High Quality"
        confidence = 1 - min(np.mean(pred_errors) / (max(pred_errors) - min(pred_errors)), 1)
        
        messagebox.showinfo(f"{wine_type.capitalize()} Wine Prediction",
                            f"{wine_type.capitalize()} Wine Quality Score: {prediction:.2f}\n"
                            f"Quality Catagory: {quality_classification}\n"
                            f"Accuracy: {accuracy * 100:.2f}%\n"
                            f"Confidence: {confidence * 100:.2f}%")
    except Exception as e:
        messagebox.showerror("Prediction Error", f"An error occurred: {e}")

def validate_and_predict():
    wine_type = wine_type_var.get()
    if wine_type not in ["red", "white"]:
        messagebox.showerror("Wine Type Error", "Please select a wine type.")
        return
    
    input_values = [entry.get().strip() for entry in entries]
    filled_count = sum(1 for value in input_values if value)
    if filled_count < 4:
        messagebox.showerror("Input Error", "Please enter at least 4 features.")
        return
    
    out_of_range_features = []
    try:
        input_values = [float(value) if value else None for value in input_values]
        for i, value in enumerate(input_values):
            if value is not None:
                min_val, max_val = feature_ranges[feature_labels[i]]
                if not (min_val <= value <= max_val):
                    out_of_range_features.append(f"{feature_labels[i].title()}: {min_val} - {max_val}")
        
        if out_of_range_features:
            messagebox.showerror("Input Error", "Please enter valid values within the allowed range:\n" + "\n".join(out_of_range_features))
            return
        
        predict_quality(wine_type, input_values)
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numeric values.")

def clear_inputs():
    for entry in entries:
        entry.delete(0, tk.END)

def toggle_lightblue():
    current_color = root.cget("bg")
    new_color = "lightblue" if current_color != "lightblue" else "white"
    root.configure(bg=new_color)
    for widget in root.winfo_children():
        widget.configure(bg=new_color)

train_models()

root = tk.Tk()
root.title("Wine Quality Prediction")
root.configure(bg="lightblue")

wine_type_var = tk.StringVar(value="red")

tk.Label(root, text="Select Wine Type:", bg="lightblue", font=("Helvetica", 12)).grid(row=0, column=0, padx=10, pady=5, sticky="w")
tk.OptionMenu(root, wine_type_var, "red", "white").grid(row=0, column=1, padx=10, pady=5)

entries = []
feature_labels = list(feature_ranges.keys())

for i, feature in enumerate(feature_labels):
    tk.Label(root, text=feature.title(), bg="lightblue", font=("Helvetica", 12)).grid(row=i + 1, column=0, padx=10, pady=5, sticky="w")
    entry = tk.Entry(root, font=("Helvetica", 12))
    entry.grid(row=i + 1, column=1, padx=10, pady=5)
    entries.append(entry)

tk.Button(root, text="Predict Quality", command=validate_and_predict, bg="green", fg="white", font=("Helvetica", 12)).grid(row=len(feature_labels) + 1, column=0, padx=10, pady=10, columnspan=2)
tk.Button(root, text="Clear Inputs", command=clear_inputs, bg="red", fg="white", font=("Helvetica", 12)).grid(row=len(feature_labels) + 2, column=0, padx=10, pady=5, columnspan=2)
tk.Button(root, text="Toggle Light Blue Mode", command=toggle_lightblue, bg="yellow", font=("Helvetica", 12)).grid(row=len(feature_labels) + 3, column=0, padx=10, pady=5, columnspan=2)

root.mainloop()


C:\Users\user\AppData\Local\Temp\ipykernel_21972\3142855094.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filled_features = [float(input_features[i]) if input_features[i] else means[i] for i in range(len(input_features))]
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
